In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row, SparkSession

In [2]:
# load the datasets
df_books = pd.read_csv('Datasets/book1-100k.csv')
df_ratings = pd.read_csv('Datasets/user_rating_0_to_1000.csv')

In [3]:
df_books.head(1)

,Id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher,CountsOfReview,PublishYear,Language,Authors,Rating,RatingDist2,RatingDist5,ISBN,RatingDist3
0,1,Harry Potter and the Half-Blood Prince (Harry ...,1:9896,652,4:556485,total:2298124,16,9,Scholastic Inc.,28062,2006,eng,J.K. Rowling,4.57,2:25317,5:1546466,NaN,3:159960


In [4]:
df_ratings.head(1)

,ID,Name,Rating
0,1,Agile Web Development with Rails: A Pragmatic ...,it was amazing


In [5]:
df_ratings['Rating'].unique()

array(['it was amazing', 'really liked it', 'liked it', 'did not like it',
       'it was ok', "This user doesn't have any rating"], dtype=object)

In [6]:
rating_mapping = {
    "This user doesn't have any rating": 0,
    'did not like it': 1,
    'it was ok': 2,
    'liked it': 3,
    'really liked it': 4,
    'it was amazing': 5
}
    
df_ratings_with_id = pd.merge(
    left=df_ratings,
    right=df_books[['Id', 'Name']],
    how='inner',
    on='Name'
).rename(columns={'ID':'User ID', 'Id': 'Book ID'})

df_ratings_with_id['Rating'] = df_ratings_with_id['Rating'].apply(lambda x: rating_mapping[x])

df_ratings_simple = df_ratings_with_id[['User ID', 'Book ID', 'Rating']]

In [7]:
np.savetxt('rating.txt', df_ratings_simple.values, fmt='%d')

In [8]:
spark = SparkSession.builder.appName('Books Recommender System').getOrCreate()

21/12/16 13:22:01 WARN Utils: Your hostname, Idriss-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.100.16 instead (on interface en0)
21/12/16 13:22:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/16 13:22:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalAccessError: class org.apache.spark.storage.StorageUtils$ (in unnamed module @0x35b74c5c) cannot access class sun.nio.ch.DirectBuffer (in module java.base) because module java.base does not export sun.nio.ch to unnamed module @0x35b74c5c
	at org.apache.spark.storage.StorageUtils$.<init>(StorageUtils.scala:213)
	at org.apache.spark.storage.StorageUtils$.<clinit>(StorageUtils.scala)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.<init>(BlockManagerMasterEndpoint.scala:110)
	at org.apache.spark.SparkEnv$.$anonfun$create$9(SparkEnv.scala:348)
	at org.apache.spark.SparkEnv$.registerOrLookupEndpoint$1(SparkEnv.scala:287)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:336)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:191)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:277)
	at org.apache.spark.SparkContext.<init>(SparkContext.s